Using the PuLP library to implement supply chain optimization 

In [ ]:
import pandas as pd
from pulp import *

Load the dataset

In [ ]:
data = pd.read_csv("supply_chain_data.csv")

Create a list of nodes (locations)


In [ ]:
locations = list(set(data['location']))

Create a dictionary of shipping costs between locations

In [ ]:

shipping_costs = {}
for i in range(len(data)):
    origin = data.iloc[i]['origin']
    destination = data.iloc[i]['destination']
    cost = data.iloc[i]['freight_cost']
    shipping_costs[(origin, destination)] = cost

Create a dictionary of warehouse costs for each location

In [ ]:

warehouse_costs = {}
for i in range(len(data)):
    location = data.iloc[i]['location']
    cost = data.iloc[i]['warehousing_cost']
    warehouse_costs[location] = cost

Define the optimization problem

In [ ]:
prob = LpProblem("Supply Chain Optimization", LpMinimize)

Create decision variables for shipping between locations

In [ ]:

shipping_vars = LpVariable.dicts("Shipping", [(o,d) for o in locations for d in locations if o!=d], 
                                lowBound=0, cat='Continuous')

Create decision variables for warehouse inventory at each location

In [ ]:

inventory_vars = LpVariable.dicts("Inventory", locations, lowBound=0, cat='Continuous')

Create the objective function

In [ ]:
prob += lpSum([shipping_costs[(o,d)]*shipping_vars[(o,d)] for o in locations for d in locations if o!=d]) \
        + lpSum([warehouse_costs[l]*inventory_vars[l] for l in locations])

Add the constraints

In [ ]:
for l in locations:
    # The sum of inbound and outbound shipping should equal inventory
    prob += lpSum([shipping_vars[(o,l)] for o in locations if o!=l]) \
            - lpSum([shipping_vars[(l,d)] for d in locations if d!=l]) == inventory_vars[l]
    # Inventory at each location should not exceed the maximum capacity
    prob += inventory_vars[l] <= data.loc[data['location'] == l, 'max_capacity'].iloc[0]

Solve the problem

In [ ]:
prob.solve()

Print the results

In [ ]:

print("Optimal solution found with total cost:", value(prob.objective))
print("Shipping plan:")
for o in locations:
    for d in locations:
        if o!=d:
            if shipping_vars[(o,d)].varValue > 0:
                print(f"{o} -> {d}: {shipping_vars[(o,d)].varValue}")
print("Warehouse inventory:")
for l in locations:
    print(f"{l}: {inventory_vars[l].varValue}")